In [ ]:
import pandas as pd
import numpy as np
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vstepanenko/disaster-tweets")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'disaster-tweets' dataset.
Path to dataset files: /kaggle/input/disaster-tweets


In [ ]:
import pandas as pd
df= pd.read_csv("/root/.cache/kagglehub/datasets/vstepanenko/disaster-tweets/versions/3/tweets.csv")

In [ ]:
data = df['text'].astype(str).tolist()[:500]

In [ ]:
def clean_text(text):
    text = "".join(v for v in text if v not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii", 'ignore')
    return text

In [ ]:
import string
corpus = [clean_text(x) for x in data]

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [ ]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [ ]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    LSTM(150, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print("Starting training on Disaster Tweets...")
model.fit(X, y, epochs=50, verbose=1)

Starting training on Disaster Tweets...
Epoch 1/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 19s 79ms/step - accuracy: 0.0460 - loss: 6.7486
Epoch 2/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.0461 - loss: 6.6335
Epoch 3/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 0.0486 - loss: 6.5316
Epoch 4/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 22s 86ms/step - accuracy: 0.0552 - loss: 6.4483
Epoch 5/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 82ms/step - accuracy: 0.0539 - loss: 6.3061
Epoch 6/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 20s 80ms/step - accuracy: 0.0565 - loss: 6.2298
Epoch 7/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step - accuracy: 0.0622 - loss: 6.1012
Epoch 8/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 19s 80ms/step - accuracy: 0.0645 - loss: 6.0038
Epoch 9/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 21s 87ms/step - accuracy: 0.0671 - loss: 5.8904
Epoch 10/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 41s 87ms/step - accuracy: 0.0734 - loss: 5.7382
Epoch 11/50
240/240 ━━━━━━━━━━━━━━━━━━━━ 19s 79ms/step - accuracy: 0.0783 - los

In [ ]:
def generate_tweet_text(seed_text, next_words=3):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text


In [ ]:
print("\n--- Model Predictions ---")
print(generate_tweet_text("Forest fire", next_words=5))
print(generate_tweet_text("Emergency", next_words=4))


--- Model Predictions ---
Forest fire in gauteng people go to
Emergency sets cars ablaze at
